# Extra Aufgaben

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
LOCAL = False

In [3]:
if LOCAL:
  PATH = "./data"
else:
  PATH = "gdrive/MyDrive/application-project-abgabe"

## Filterung der Datenbank um irrelevante Ergebnisse zu verhindern

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel, polynomial_kernel, sigmoid_kernel, rbf_kernel, laplacian_kernel, chi2_kernel, euclidean_distances, manhattan_distances, cosine_distances
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm
import time
from ast import literal_eval

In [5]:
#by_isrc = pd.read_csv('data/by_isrc.csv')
by_isrc = pd.read_csv(f"{PATH}/by_isrc.csv")

In [ ]:
by_isrc["country_code"] = by_isrc["isrc"].apply(lambda x: x[:2])

In [ ]:
by_isrc["country_code"].unique()

array(['AE', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BC', 'BE', 'BG', 'BR', 'BX',
       'Br', 'CA', 'CH', 'CI', 'CL', 'CN', 'CO', 'CS', 'CZ', 'DE', 'DG',
       'DK', 'EE', 'EG', 'ES', 'FI', 'FR', 'FX', 'GB', 'GN', 'GR', 'GT',
       'GX', 'HK', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IR', 'IS', 'IT',
       'JM', 'JP', 'KE', 'KR', 'KS', 'LB', 'LK', 'LT', 'LU', 'LV', 'MD',
       'MM', 'MT', 'MU', 'MX', 'MY', 'NG', 'NI', 'NL', 'NO', 'NZ', 'No',
       'PE', 'PH', 'PL', 'PT', 'QM', 'QZ', 'RE', 'RO', 'RS', 'RU', 'SE',
       'SG', 'SI', 'SK', 'SM', 'SW', 'TC', 'TH', 'TR', 'TW', 'UA', 'UK',
       'UR', 'US', 'UY', 'VE', 'VN', 'ZA', 'ZZ', 'ae', 'ca', 'gb', 'il',
       'lv', 'nl', 'qz', 'us', 've'], dtype=object)

Einige Ländercodes sind kleingeschrieben. Diese sollen in Großbuchstaben umgewandelt werden.

In [ ]:
by_isrc["country_code"] = by_isrc["country_code"].str.upper()

In [ ]:
by_isrc["country_code"].unique()

array(['AE', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BC', 'BE', 'BG', 'BR', 'BX',
       'CA', 'CH', 'CI', 'CL', 'CN', 'CO', 'CS', 'CZ', 'DE', 'DG', 'DK',
       'EE', 'EG', 'ES', 'FI', 'FR', 'FX', 'GB', 'GN', 'GR', 'GT', 'GX',
       'HK', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IR', 'IS', 'IT', 'JM',
       'JP', 'KE', 'KR', 'KS', 'LB', 'LK', 'LT', 'LU', 'LV', 'MD', 'MM',
       'MT', 'MU', 'MX', 'MY', 'NG', 'NI', 'NL', 'NO', 'NZ', 'PE', 'PH',
       'PL', 'PT', 'QM', 'QZ', 'RE', 'RO', 'RS', 'RU', 'SE', 'SG', 'SI',
       'SK', 'SM', 'SW', 'TC', 'TH', 'TR', 'TW', 'UA', 'UK', 'UR', 'US',
       'UY', 'VE', 'VN', 'ZA', 'ZZ'], dtype=object)

Gruppierung der Songs nach den Ländercodes.

In [ ]:
countries = {
    'AE': 'United Arab Emirates',
    'AR': 'Argentina',
    'AT': 'Austria',
    'AU': 'Australia',
    'AZ': 'Azerbaijan',
    'BA': 'Bosnia and Herzegovina',
    'BC': 'Canada (British Columbia)',
    'BE': 'Belgium',
    'BG': 'Bulgaria',
    'BR': 'Brazil',
    'BX': 'Benelux',
    'CA': 'Canada',
    'CH': 'Switzerland',
    'CI': 'Ivory Coast (Côte d\'Ivoire)',
    'CL': 'Chile',
    'CN': 'China',
    'CO': 'Colombia',
    'CS': 'Serbia and Montenegro',
    'CZ': 'Czech Republic',
    'DE': 'Germany',
    'DG': 'Germany (East)',
    'DK': 'Denmark',
    'EE': 'Estonia',
    'EG': 'Egypt',
    'ES': 'Spain',
    'FI': 'Finland',
    'FR': 'France',
    'FX': 'France (Metropolitan)',
    'GB': 'United Kingdom',
    'GN': 'Guinea',
    'GR': 'Greece',
    'GT': 'Guatemala',
    'GX': 'Greece (Ancient)',
    'HK': 'Hong Kong',
    'HR': 'Croatia',
    'HU': 'Hungary',
    'ID': 'Indonesia',
    'IE': 'Ireland',
    'IL': 'Israel',
    'IN': 'India',
    'IR': 'Iran',
    'IS': 'Iceland',
    'IT': 'Italy',
    'JM': 'Jamaica',
    'JP': 'Japan',
    'KE': 'Kenya',
    'KR': 'South Korea',
    'KS': 'Kosovo',
    'LB': 'Lebanon',
    'LK': 'Sri Lanka',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'LV': 'Latvia',
    'MD': 'Moldova',
    'MM': 'Myanmar (Burma)',
    'MT': 'Malta',
    'MU': 'Mauritius',
    'MX': 'Mexico',
    'MY': 'Malaysia',
    'NG': 'Nigeria',
    'NI': 'Nicaragua',
    'NL': 'Netherlands',
    'NO': 'Norway',
    'NZ': 'New Zealand',
    'PE': 'Peru',
    'PH': 'Philippines',
    'PL': 'Poland',
    'PT': 'Portugal',
    'QM': 'Marshall Islands', #second country code for the United States
    'QZ': 'Unknown or Invalid Area',
    'RE': 'Réunion',
    'RO': 'Romania',
    'RS': 'Serbia',
    'RU': 'Russia',
    'SE': 'Sweden',
    'SG': 'Singapore',
    'SI': 'Slovenia',
    'SK': 'Slovakia',
    'SM': 'San Marino',
    'SW': 'Sweden',
    'TC': 'Turks and Caicos Islands',
    'TH': 'Thailand',
    'TR': 'Turkey',
    'TW': 'Taiwan',
    'UA': 'Ukraine',
    'UK': 'United Kingdom',
    'UR': 'Uruguay',
    'US': 'United States',
    'UY': 'Uruguay',
    'VE': 'Venezuela',
    'VN': 'Vietnam',
    'ZA': 'South Africa',
    'ZZ': 'Unknown or Invalid Area'
}

Erstellen eines neuen Features `country`, welches den Namen des jeweiligen Landes beinhaltet.

In [ ]:
by_isrc["country"] = by_isrc["country_code"].apply(lambda x: countries[x])

Gruppieren der Daten nach `country` und `country_code`. Dies soll genutzt werden, um zu analysieren, aus welchem Land am meisten Songs veröffentlicht worden sind.

In [ ]:
grouped_countries = by_isrc.copy()
grouped_countries = grouped_countries[["country_code", "country"]]
grouped_countries["num_countries"] = 0
grouped_countries = grouped_countries.groupby(["country_code", "country"]).count()

In [ ]:
grouped_countries = grouped_countries.sort_values("num_countries", ascending=False)

In [ ]:
grouped_countries.reset_index(inplace=True)

In [ ]:
# country und country_code werden kombiniert für eine bessere Darstellung
grouped_countries["country_comb"] = grouped_countries.apply(lambda x: f'{x["country"]}-{x["country_code"]}', axis=1)

In [ ]:
fig = px.bar(grouped_countries[:30], x="country_comb", y="num_countries", template="plotly_dark")
fig.update_layout(
    height=500,
    title="Veröffentlichte Songs pro Land (Top 30)",
    xaxis_title="Land",
    yaxis_title="Veröffentlichte Songs"
)
fig.show()

Es sollen alle Lieder entfernt werden welche nicht aus englischsprachigen Länger (US, UK, CA, AU, Neuseeland) sowie Deutschland, Schweiz, Österreich, Italien und Schweden stammen.

In [ ]:
important_country_codes = ["US", "GB", "CA", "AU", "NZ", "DE", "CH", "AT", "IT", "SE"]

# kleinere Inselgegenden von Amerika: Baker Island, Howland Island, Jarvis Island, Johnston Atoll, Kingman Reef, Midway Islands, Navassa Island, Palmyra Atoll, Wake Island
minor_insular_areas_us = ["XB", "XH", "XQ", "XU", "XM", "QM", "XV", "XL", "QW"]
important_country_codes.extend(minor_insular_areas_us)

`QM` wird heute als zusätzlicher Country code für Amerika verwendet, da der Code `US` sein Limit erreicht hat. Da dieser Beschluss jedoch aus 2010 stammt, sollten die Lieder der 80er nicht beeinflusst werden?

In [ ]:
by_isrc[by_isrc["country_code"] == "QM"].sort_values("popularity", ascending=False)[["isrc", "genres", "name", "artists", "popularity", "chart_power"]]

,isrc,genres,name,artists,popularity,chart_power
274154,QMKHM1600219,"['hard-rock', 'metal', 'rock']",master of puppets,metallica,80,NaN
274107,QMKHM1600096,"['hard-rock', 'metal', 'rock']",for whom the bell tolls - remastered,metallica,75,NaN
271243,QMFME1326440,"['latin', 'pop']",tu dama de hierro,marisela,71,NaN
274108,QMKHM1600097,"['hard-rock', 'metal', 'rock']",fade to black - remastered,metallica,70,NaN
274057,QMKHM1600034,"['hard-rock', 'metal', 'rock']",seek & destroy - remastered,metallica,69,NaN
...,...,...,...,...,...,...
267541,QM7281628419,['tango'],fuegos artificiales,juan d'arienzo y su orquesta típica,0,NaN
267551,QM7281713701,['jazz'],raincheck,nick brignola,0,NaN
267552,QM7281713702,['jazz'],tenderly,nick brignola,0,NaN
267553,QM7281713703,['jazz'],hurricane connie,nick brignola,0,NaN


Herausfiltern der Lieder aus den "wichtigen" Länder.

In [ ]:
reduced_isrc = by_isrc.copy()
reduced_isrc = reduced_isrc[reduced_isrc["country_code"].isin(important_country_codes)]
reduced_isrc["genres"] = reduced_isrc["genres"].apply(literal_eval)

In [ ]:
print(f"Die Anzahl der Lieder hat sich von {by_isrc.shape[0]} auf {reduced_isrc.shape[0]} verringert.")

Die Anzahl der Lieder hat sich von 416154 auf 251101 verringert.


Gruppieren der Daten nach den Genres => Welche Genres waren in den wichtigen Ländern relevant?

In [ ]:
def filter_important_genres(df):
  important_genres = df.copy()
  important_genres = important_genres.explode("genres")
  important_genres["num"] = 0
  important_genres = important_genres[["genres", "num"]]
  important_genres = important_genres.groupby("genres").count()
  important_genres.reset_index(inplace=True)
  important_genres = important_genres.sort_values("num", ascending=False)
  return important_genres

In [ ]:
filter_important_genres(reduced_isrc)

,genres,num
16,classical,63432
87,rock,60598
77,pop,48841
38,german,28985
62,jazz,22632
...,...,...
28,dubstep,2
12,cantopop,1
53,idm,1
69,metalcore,1


Am meisten Lieder wurden für das Genre Klassik veröffentlicht. Da dieses Genre nicht wirklich relevant ist, müssen zunächst die irrelevanten Genres entfernt werden. Hiebei gibt es zwei verschiedene Wege. Verfügt ein Künstler sowohl über ein erwünschtes als auch ein unerwünschtes Genre, so können entweder alle Songs von ihm behalten oder entfernt werden. Zunächst wird geprüft, wie häufig ein solcher Fall eintritt.

In [ ]:
unwanted_genres = ["classical", "jazz", "folk", "french", "turkish", "gospel", "samba", "piano", "mpb", "sertanejo", "pagode", "sleep", "forro", "malay", "anime",
                 "j-idol", "comedy", "mandopop", "cantopop", "show-tunes", "emo", "romance", "j-dance", "chill", "world-music", "iranian", "idm", "metalcore",
                 "hardstyle", "opera", "k-pop", "j-pop", "j-rock", "happy"]

In [ ]:
occ_unwanted_and_wanted = by_isrc.copy()
occ_unwanted_and_wanted = occ_unwanted_and_wanted[["genres"]]
occ_unwanted_and_wanted["genres"] = occ_unwanted_and_wanted["genres"].apply(literal_eval)
occ_unwanted_and_wanted = occ_unwanted_and_wanted.explode("genres")

In [ ]:
wanted_genres = list(set(occ_unwanted_and_wanted["genres"].unique()).difference(unwanted_genres))

In [ ]:
def filter_row(row):
    has_wanted_genre = any(genre in row['genres'] for genre in wanted_genres)
    has_not_wanted_genre = any(genre in row['genres'] for genre in unwanted_genres)
    return has_wanted_genre and has_not_wanted_genre

In [ ]:
occ_unwanted_and_wanted_df = reduced_isrc[reduced_isrc.apply(filter_row, axis=1)]

In [ ]:
print(f"Insgesamt gibt es {occ_unwanted_and_wanted_df.shape[0]} Lieder, welche über ein erwünschtes und ein unerwünschtes Genre verfügen.")

Insgesamt gibt es 48318 Lieder, welche über ein erwünschtes und ein unerwünschtes Genre verfügen.


In [ ]:
print(f"Hierbei gibt es {len(list(occ_unwanted_and_wanted_df.genres.astype(str).unique()))} verschiedene Kombinationen der Genres.")

Hierbei gibt es 582 verschiedene Kombinationen der Genres.


## Visualisierung Kombination

Es werden alle Songs entfernt, welche mindestens ein unerwünschtes Feature beinhalten.

In [ ]:
by_isrc_sub = by_isrc.copy()
by_isrc_sub = by_isrc_sub[['isrc', 'genres']]
by_isrc_sub = by_isrc_sub[~by_isrc_sub['genres'].str.contains('|'.join(unwanted_genres))]
by_isrc_sub['genres'] = by_isrc_sub['genres'].apply(literal_eval)

Das neu entstandene DataFrame soll nun mit dem bereits auf die wichtigen Länder reduzierten DataFrame gemerged werden.

In [ ]:
#reduced_isrc.drop(columns="genres", inplace=True)

In [ ]:
reduced_isrc.shape, by_isrc_sub.shape

((251101, 27), (220702, 2))

In [ ]:
reduced_isrc = reduced_isrc.merge(by_isrc_sub[["isrc"]], how="inner", on="isrc")

In [ ]:
print(f"Die Anzahl der Songs hat sich weiter auf {reduced_isrc.shape[0]} verringert. Die Anzahl der Features ist bei {reduced_isrc.shape[1]} geblieben.")

Die Anzahl der Songs hat sich weiter auf 139629 verringert. Die Anzahl der Features ist bei 27 geblieben.


In [ ]:
important_genres = filter_important_genres(reduced_isrc)
important_genres = important_genres.sort_values("num", ascending=False)
important_genres.head(10)

,genres,num
61,rock,52805
51,pop,43525
56,punk,16552
14,country,12353
47,metal,11743
35,hard-rock,9228
29,german,8980
20,disco,8065
70,swedish,7964
68,soul,7289


In [ ]:
fig = px.bar(important_genres[:30], x="genres", y="num", template="plotly_dark")
fig.update_layout(
    height=500,
    title="Veröffentlichte Songs pro Genre in den wichtigen Ländern (Top 30)",
    xaxis_title="Genre",
    yaxis_title="Veröffentlichte Songs"
)
fig.show()

In [ ]:
reduced_isrc.to_csv(f"{PATH}/reduced_isrc.csv")

## Auffindbarkeiten prüfen (in db und auf Spotify)

In [6]:
!pip install colab-env --upgrade

  Preparing metadata (setup.py) ... done
  Created wheel for colab-env: filename=colab_env-0.2.0-py3-none-any.whl size=3805 sha256=a3c1482a31faf30b67da4108ec3db9acddcad04b1f6139acad39eeac0281c879
  Stored in directory: /root/.cache/pip/wheels/ae/36/4f/466c2cd4db5d08f317893a920c4a0f58a81459ee3bdb136d35
Successfully built colab-env


In [7]:
!pip install spotipy --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 7.2 MB/s eta 0:00:00


In [8]:
!pip install ratelimit

  Preparing metadata (setup.py) ... done
  Created wheel for ratelimit: filename=ratelimit-2.2.1-py3-none-any.whl size=5894 sha256=2b309c0c9f9178cf6e11b6c1a70f961e34f96ca5d1bd1b844ce76ba05c06756f
  Stored in directory: /root/.cache/pip/wheels/27/5f/ba/e972a56dcbf5de9f2b7d2b2a710113970bd173c4dcd3d2c902
Successfully built ratelimit


In [9]:
import colab_env
import os
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
import json
from numpy import NaN
from tqdm import tqdm
import time
from ratelimit import limits, sleep_and_retry

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
@sleep_and_retry
@limits(calls=60, period=30)
def call_api():
  return

In [11]:
chart_power_df = pd.read_excel(f"{PATH}/chart-power-scores_80s.xlsx")
chart_power_df = chart_power_df.applymap(lambda s: s.lower() if type(s) == str else s)
chart_power_df = chart_power_df[['Song', 'Artist', 'Points']].groupby(['Song', 'Artist']).sum()
chart_power_df.reset_index(inplace=True)

In [12]:
if LOCAL:
    from dotenv import load_dotenv
    load_dotenv()
    SPOTIFY_CLIENT_ID = os.environ.get("SPOTIFY_CLIENT_ID")
    SPOTIFY_CLIENT_SECRET = os.environ.get("SPOTIFY_CLIENT_SECRET")
else:
    SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
    SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

In [13]:
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Abfrage aller Lieder für Boney M.

In [14]:
def query(year: int, offset: int):
  res = sp.search(q=f"artist:Boney M. year:{year}", type="track", limit=10, offset=offset, market="DE")
  if (len(res["tracks"]["items"]) == 10):
    return query(year, offset + 10)
  return len(res["tracks"]["items"]) + offset

In [ ]:
boney_songs = pd.DataFrame()
for year in range(1980, 1990):
  boney_songs = pd.concat([boney_songs, pd.DataFrame([[year, query(year, 0)]], columns=["year", "songs"])])

In [ ]:
fig = px.bar(boney_songs, x="year", y="songs", title="Songs veröffentlich von Boney M. in den 80er Jahren")
fig.update_layout(
    height=500,
    template='plotly_dark',
    xaxis_title="Jahr",
    yaxis_title="Veröffentlichte Songs"
)
fig.show()

Einbauen des Genres in die query

In [ ]:
def genre_query(artist: str, year: int, offset:int, genres):
  for genre in genres:
    call_api()
    res = sp.search(q=f"artist:{artist} year:{year} genre:{genre}", type="track", limit=50, offset=offset, market="DE")
    if (len(res["tracks"]["items"] ) > 1):
      print(year, genre)
      #print(res)

In [ ]:
def test_genre_query(artist: str):
  genres = sp.recommendation_genre_seeds()['genres']
  for year in range(1980, 1990):
    print(year)
    genre_query(artist, year, 0, genres)

In [ ]:
test_genre_query("Boney M.")

1980


KeyboardInterrupt: 

Es existieren keine Songs für Boney M. wenn ein Genre in der Abfrage spezifiziert wird. Eventuell sind die Genres das Problem für die fehlenden Daten.

In [ ]:
by_isrc.sort_values("popularity", ascending=False)

,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,chart_power,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,country_code,country
132734,GBAAM0201110,['rock'],every breath you take,the police,Synchronicity (Remastered 2003),1983-06-17,day,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,1JSTJqkT5qHq8MDJnJbRE1,1419.0,...,0.5430,0.002940,0.0714,0.7400,117.401,253920,4,1983,GB,United Kingdom
144156,GBALX8300190,"['piano', 'rock']",i'm still standing,elton john,Too Low For Zero,1983-05-30,day,spotify:track:1jDJFeK9x3OZboIAHsY9k2,1jDJFeK9x3OZboIAHsY9k2,1185.0,...,0.3560,0.121000,0.1400,0.7720,176.808,183440,4,1983,GB,United Kingdom
368082,USPR38619998,"['metal', 'rock']",livin' on a prayer,bon jovi,Slippery When Wet,1986-08-16,day,spotify:track:37ZJ0p5Jm13JPevGcx4SkF,37ZJ0p5Jm13JPevGcx4SkF,1112.0,...,0.0778,0.000214,0.2940,0.7950,122.511,249293,4,1986,US,United States
169272,GBCNR8500002,"['piano', 'pop', 'r-n-b', 'rock', 'singer-song...",running up that hill (a deal with god),kate bush,Hounds Of Love,1985-09-16,day,spotify:track:1PtQJZVZIdWIYdARpZRDFO,1PtQJZVZIdWIYdARpZRDFO,NaN,...,0.7190,0.003080,0.0604,0.1940,108.376,298933,4,1985,GB,United Kingdom
6804,AUAP08000046,"['hard-rock', 'rock']",back in black,ac/dc,Back In Black,1980-07-25,day,spotify:track:08mG3Y1vljYA6bvDt4Wqkj,08mG3Y1vljYA6bvDt4Wqkj,NaN,...,0.0110,0.009650,0.0828,0.7630,188.386,255493,4,1980,AU,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174547,GBF078300092,['classical'],"serenade in g, k.525 ""eine kleine nachtmusik"" ...","wolfgang amadeus mozart,salomon quartet,barry guy",Mozart: Eine kleine Nachtmusik; Notturno; Sere...,1984-06-03,day,spotify:track:1Sbp0G9AUAQdhXinEiDWsl,1Sbp0G9AUAQdhXinEiDWsl,NaN,...,0.5170,0.000000,0.2780,0.1510,124.043,241373,3,1984,GB,United Kingdom
174528,GBF078300017,['classical'],"alexander nevsky, op. 78: 7. alexander's entry...","sergei prokofiev,the cleveland orchestra choru...",Prokofiev: Alexander Nevsky,1984-01-01,day,spotify:track:3EZH0BH50us07EUPKzhw3u,3EZH0BH50us07EUPKzhw3u,NaN,...,0.9160,0.003540,0.1860,0.1460,145.809,277053,4,1984,GB,United Kingdom
174527,GBF078300016,['classical'],"alexander nevsky, op. 78: 6. field of the dead","sergei prokofiev,irina arkhipova,cleveland orc...",Prokofiev: Alexander Nevsky,1984-01-01,day,spotify:track:4EXE7ZRcKVwD1xxNZLEOPj,4EXE7ZRcKVwD1xxNZLEOPj,NaN,...,0.9160,0.004890,0.1180,0.0399,83.968,386533,4,1984,GB,United Kingdom
174526,GBF078300015,['classical'],"alexander nevsky, op. 78: 5. the battle on the...","sergei prokofiev,the cleveland orchestra choru...",Prokofiev: Alexander Nevsky,1984-01-01,day,spotify:track:3jXNm9r9WAwj8qJvBKoXg1,3jXNm9r9WAwj8qJvBKoXg1,NaN,...,0.6560,0.787000,0.1470,0.0637,100.909,801840,4,1984,GB,United Kingdom


Abfrage nach dem Künstler Elton John.

In [ ]:
test_genre_query("Elton John")

1980 piano
1980 rock
1981 piano
1981 rock
1982 piano
1982 rock
1983 piano
1983 rock


KeyboardInterrupt: ignored

Für Elton John existieren Genres. Im nächsten Schritt soll überprüft werden, ob die Genres auch in dem Response der Spotify API wiedergefunden wird.

In [ ]:
res = sp.search(q="artist:Elton John", type="track", market="DE", limit=50, offset=0)
res = res["tracks"]
if "items" in res:
  for item in res["items"]:
    if "artists" in item:
      #print(item["artists"]["genres"] if "genres" in item["artists"] else print("No genres available!"))
      #print(item["artists"])
      for artist in item["artists"]:
        print(artist["genres"] if "genres" in artist else artist.keys())
    else:
      print("No artists in item")
else:
  print("No items in res")
  print(res.keys())


dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(

Die Spotify gibt auch für Künstler die ein Genre zugewiesen bekommen haben kein Genre zurück. Laut API sollte es ein weiteres Attribut `genres` geben.

Abfrage aller Lieder von Künstlern, welche in den 80ern in den Charts waren

In [23]:
chart_power_df["Artist"].unique()

array(['paul hardcastle', 'c. c. catch', 'deborah sasson & mcl',
       'coast to coast', 'fiction factory', 'o.m.d.',
       'the rock steady crew', 'cutting crew', 'sandra',
       'bill medley & jennifer warnes', 'john lennon', 'beastie boys',
       'klaus lage band', 'schweizer', 'sigue sigue sputnik',
       'jennifer rush', 'the catch', 'dolly parton', 'nena',
       'george michael', 'feargal sharkey', 'phil collins', 'queen',
       'godley & creme', 'erasure', 'sheila e', 'will downing',
       "shakin' stevens", 'depeche mode',
       "shakin' stevens & bonnie tyler", 'duran duran',
       'nick straker band', 'bad boys blue', 'genesis',
       'steve miller band', 'katja ebstein', 'roger whittaker',
       'david bowie', 'andy borg', 'toto', 'rose laurens', 'princess',
       'saragossa band', 'john farnham', 'ricky king', 'quincy jones',
       'bill withers', 'rufus & chaka khan', 'inner city', 'radiorama',
       'simple minds', 'herbert grã¶nemeyer', 'sister sledge',
  

Wenn mehrere Künstler zusammen einen Song veröffentlich haben, sind diese hier mit einem `&` getrennt. Dies muss zunächst aufgelöst werden.

In [15]:
chart_power_df["Artist"] = chart_power_df["Artist"].str.split(" & ")
chart_power_df_explode = chart_power_df.explode("Artist", ignore_index=True)

In [16]:
chart_power_df_explode

,Song,Artist,Points
0,19,paul hardcastle,1600
1,'cause you're young,c. c. catch,998
2,(carmen) danger in her eyes,deborah sasson,531
3,(carmen) danger in her eyes,mcl,531
4,(do) the hucklebuck,coast to coast,697
...,...,...,...
2022,zu spät,die ã„rzte,605
2023,zuppa romana,schrott nach 8,601
2024,â€¦ und ganz doll mich (ich mag),rolf und seine freunde,1314
2025,über sieben brücken mußt du geh'n,karat,826


Einige Künstler und Lieder enthalten zudem Umlaute, welche zunächst entsprechend formatiert werden müssen (z.B. `Die Ärzte` sind als `Die ã„rzte` gespeichert).

In [17]:
replace = {
    "ã¶": "ö",
    "ã„": "ä",
    "ã–": "ö",
    "ã©": "e"
}

chart_power_df_explode["Song"] = chart_power_df_explode["Song"].replace(replace, regex=True)
chart_power_df_explode["Artist"] = chart_power_df_explode["Artist"].replace(replace, regex=True)

In [18]:
chart_power_df_explode[chart_power_df_explode["Song"] == "zu spät"]

,Song,Artist,Points
2022,zu spät,die ärzte,605


In [19]:
chart_power_df_explode[chart_power_df_explode["Artist"] == "die ã„rzte"]


,Song,Artist,Points


In [20]:
chart_power_df_explode[chart_power_df_explode["Artist"].str.contains("\[")]

,Song,Artist,Points
64,all of me (boy oh boy),sabrina [it],841
85,amoureux solitaires,lio [be],1555
100,another life,kano [it],1460
187,boys (summertime love),sabrina [it],1444
198,bridge to your heart,wax [uk],614
256,change your mind,raff [it],736
314,cry softly,secret service [se],451
351,delirio mind,scotch [it],1120
390,disco band,scotch [it],1806
480,ein weißes blatt'l papier,relax [de],535


Weiterhin ist hinter einigen Künstlern das Herukunftsland in eckigen Klammern gekennzeichnet. Dies muss ebenfalls entfernet werden, da diese andernfalls nicht von der Spotify API gefunden werden.

In [21]:
pattern = r' \[.*?\]'
chart_power_df_explode["Artist"] = chart_power_df_explode["Artist"].str.replace(pattern, '',regex=True)

In [22]:
chart_power_df_explode[chart_power_df_explode["Artist"].str.contains("\[")]

,Song,Artist,Points


In [23]:
len(chart_power_df_explode["Artist"].unique())

944

In [24]:
import time

def sp_request(req, retries, max_retries, *args):
  try:
    call_api()
    return req(*args)
  except Exception as e:
    if isinstance(e, spotipy.exceptions.SpotifyException) and e.http_status == 429:
      # Handle rate limiting (HTTP 429)
      if retries < max_retries:
          sleep_time = exponential_backoff(retries)
          print(f"Rate limited, waiting for {sleep_time} seconds before retrying...")
          time.sleep(sleep_time)
          retries += 1
          return sp_request(req, retries, max_retries, *args)
      else:
          print("Max retries reached, exiting...")
    else:
      print(f"An error occurred: {e}")
      retries += 1
      return sp_request(req, retries, max_retries, *args)

# Define a function for exponential backoff
def exponential_backoff(retries):
    return 2 ** retries

In [25]:
def filter_track_features(track, features, genre):
    '''
    Filters the relevant features of a track in returns them in JSON object.

    Parameter
    ---------
    track: Object
        Track returend by the spotify API

    genre: string
        Genre that should be used

    Return
    ------
    relevant_features: Object
        JSON Object that contains the relevant featues
    '''

    external_ids = track['external_ids'] if 'external_ids' in track else {}
    isrc = external_ids['isrc'] if 'isrc' in external_ids else np.nan
    artist_names = []

    if 'artists' in track and isinstance(track['artists'],list):
        for artist in track['artists']:
            if 'name' in artist:
                artist_names.append(artist['name'])
        artist_names = ','.join(artist_names)
    else:
        artist_names = np.nan



    if 'album' in track:
        album = track['album']['name'] if 'name' in track['album'] else np.nan
        release_date = track['album']['release_date'] if 'release_date' in track['album'] else np.nan
        release_date_precision = track['album']['release_date_precision'] if 'release_date_precision' in track['album'] else np.nan
    else:
         album = np.nan
         release_date = np.nan
         release_date_precision = np.nan


    track_name = track['name'] if 'name' in track else np.nan
    if track_name != np.nan and artist_names != np.nan:
        points = chart_power_df_explode.loc[(chart_power_df_explode.Song == track_name.lower()) & (chart_power_df_explode.Artist.str.contains(artist_names.lower()))]['Points']
        if points.empty:
             points = np.nan
        else:
            try:
                points = int(points)
            except Exception as e:
                print(e)
                print(track_name)
                print(artist_names)
                print(points)
                print("-------")
    else:
         points = np.nan

    return {
        'name': track_name,
        'artists': artist_names,
        'album': album,
        'release_date': release_date,
        'release_date_precision': release_date_precision,
        'spotify_id': track['id'] if 'id' in track else np.nan,
        'chart_power': points,
        'uri': track['uri'] if 'uri' in track else np.nan,
        'popularity': track['popularity'] if 'popularity' in track else np.nan,
        'genres': np.nan if genre == False else genre,
        'danceability': features['danceability'] if 'danceability' in features else np.nan,
        'energy': features['energy'] if 'energy' in features else np.nan,
        'key': features['key'] if 'key' in features else np.nan,
        'loudness': features['loudness'] if 'loudness' in features else np.nan,
        'mode': features['mode'] if 'mode' in features else np.nan,
        'speechiness': features['speechiness'] if 'speechiness' in features else np.nan,
        'acousticness': features['acousticness'] if 'acousticness' in features else np.nan,
        'instrumentalness': features['instrumentalness'] if 'instrumentalness' in features else np.nan,
        'liveness': features['liveness'] if 'liveness' in features else np.nan,
        'valence': features['valence'] if 'valence' in features else np.nan,
        'tempo': features['tempo'] if 'tempo' in features else np.nan,
        'duration_ms': features['duration_ms'] if 'duration_ms' in features else np.nan,
        'time_signature': features['time_signature'] if 'time_signature' in features else np.nan,
        'isrc': isrc,
    }

In [26]:
def get_number_of_tracks(release_year, artist, genre):
    '''
    Retrieves the number of tracks the spotfiy API returns for a specific query.
    '''
    if genre == False:
      result = sp_request(lambda release_year, artist: sp.search(q=f'year:{release_year} artist:{artist}', type='track', limit=1, offset=0, market='DE'), 0, 100, release_year, artist)
    else:
      result = sp_request(lambda release_year, artist, genre: sp.search(q=f'year:{release_year} artist:{artist} genre:{genre}', type='track', limit=1, offset=0, market='DE'), 0, 100, release_year, artist, genre)

    tracks = result['tracks'] if 'tracks' in result else ''
    return tracks['total'] if 'total' in tracks else 0

In [27]:
genres = ['chicago-house', 'hard-rock', 'rock', 'power-pop', 'guitar', 'hardcore',
          'groove', 'songwriter', 'garage', 'disco', 'grunge', 'ambient',
          'minimal-techno', 'brazil', 'indie', 'detroit-techno', 'electronic',
          'edm', 'dubstep', 'children', 'drum-and-bass', 'indian', 'death-metal',
          'club', 'salsa', 'bluegrass', 'metal', 'reggae', 'tango', 'black-metal',
          'british', 'german', 'punk', 'reggaeton', 'alternative', 'soul', 'r-n-b',
          'goth', 'pop-film', 'breakbeat', 'indie-pop', 'heavy-metal', 'dance',
          'honky-tonk', 'dancehall', 'dub', 'singer-songwriter', 'spanish',
          'deep-house', 'rock-n-roll', 'techno', 'hip-hop', 'punk-rock', 'industrial',
          'afrobeat', 'trip-hop', 'funk', 'blues', 'swedish', 'latino', 'alt-rock',
          'country', 'acoustic', 'trance', 'grindcore', 'ska', 'house', 'progressive-house',
          'new-age', 'electro', 'rockabilly', 'party', 'pop', 'synth-pop', 'latin',
          'psych-rock']

In [173]:
'''
artist_over_1000 = []
df = pd.DataFrame()
with tqdm(total=len(genres)*len(chart_power_df_explode["Artist"].unique())) as pbar:
  for artist in chart_power_df_explode["Artist"].unique():
    for genre in genres:
      total_results = get_number_of_tracks("1980-1989", artist, genre)
      if total_results < 1000:
        offset = 0
        while offset < total_results:
          try:
            track_features = []
            call_api()
            result = sp.search(q=f"year:1980-1989 artist:{artist} genre:{genre}", type="track", limit=50, offset=offset, market="DE")
            tracks = result["tracks"] if "tracks" in result else ""
            if "items" in tracks:
              for track in tracks["items"]:
                features = filter_track_features(track, genre)
                track_features.append(features)
              offset += 50
              df = pd.concat([df, pd.DataFrame(track_features)], ignore_index=True)
              df.to_scv
            else:
              continue
          except Exception as e:
            print(e)
      else:
        print(f"Artist {artist} has published over 1000 songs in the 80s")
        artist_over_1000.append(artist)

      pbar.update(1)
      df.to_csv(f"{PATH}/data-new.csv")
  '''

'\nartist_over_1000 = []\ndf = pd.DataFrame()\nwith tqdm(total=len(genres)*len(chart_power_df_explode["Artist"].unique())) as pbar:\n  for artist in chart_power_df_explode["Artist"].unique():\n    for genre in genres:\n      total_results = get_number_of_tracks("1980-1989", artist, genre)\n      if total_results < 1000:\n        offset = 0\n        while offset < total_results:\n          try:\n            track_features = []\n            call_api()\n            result = sp.search(q=f"year:1980-1989 artist:{artist} genre:{genre}", type="track", limit=50, offset=offset, market="DE")\n            tracks = result["tracks"] if "tracks" in result else ""\n            if "items" in tracks:\n              for track in tracks["items"]:\n                features = filter_track_features(track, genre)\n                track_features.append(features)\n              offset += 50\n              df = pd.concat([df, pd.DataFrame(track_features)], ignore_index=True)\n              df.to_scv\n          

In [ ]:
# Initialize offset and retries
offset = 0

artist_over_1000 = []
no_tracks = []
no_features = []
artists_not_found = []


df = pd.DataFrame()
with tqdm(total=len(chart_power_df_explode["Artist"].unique())) as pbar:
  tracks_ = []
  audio_features_ = []

  for artist in chart_power_df_explode["Artist"].unique():
    total_results = get_number_of_tracks("1980-1989", artist, False)
    if total_results == 0:
      print(f"{artist} not found!")
      artists_not_found.append(artist)
    elif total_results < 1000:
      offset = 0
      while offset < total_results:
        result = sp_request(lambda artist, offset: sp.search(q=f"year:1980-1989 artist:{artist}", type="track", limit=50, offset=offset, market="DE"), 0, 100, artist, offset)
        offset += 50
        if "tracks" not in result:
          print("No tracks in result")
          continue

        tracks = result["tracks"]

        if "items" in tracks and isinstance(tracks["items"], list):
          for track in tracks["items"]:
            tracks_.append(track)
            if len(tracks_) == 100:
              audio_features_ = sp_request(lambda x: sp.audio_features([track["id"] for track in x]), 0, 100, tracks_)
              track_features = []
              if len(tracks_) != len(audio_features_):
                print("Tracks and features have not the same lengh!")
                print(len(tracks_), len(audio_features_))
              for t, af in list(zip(tracks_, audio_features_)):
                if not t:
                  print("No track!")
                  print(t)
                  print(af)
                  no_tracks.append(af)
                  continue
                if not af:
                  print("No features")
                  print(af)
                  print(t)
                  no_features.append(t)
                  continue
                features = filter_track_features(t, af, False)
                track_features.append(features)
              df = pd.concat([df, pd.DataFrame(track_features)], ignore_index=True)
              #df.to_csv(f"{PATH}/data-new-without-genres-final.csv")
              tracks_ = []
              audio_features_ = []
        else:
          continue
    else:
      print(f"Artist {artist} has published over 1000 songs in the 80s")
      artist_over_1000.append(artist)

    pbar.update(1)

problems = {
    "artist_over_1000": artist_over_1000,
    "artist_not_found": artists_not_found,
    "no_features": [track["id"] for track in no_features]
}
#with open(f"{PATH}/problems.json", "w") as file:
    #json.dump(problems, file)

In [31]:
df = pd.read_csv(f"{PATH}/data-new-without-genres-final.csv")

In [33]:
with open(f"{PATH}/problems.json", "r") as file:
    loaded_problems = json.load(file)
    artist_over_1000 = loaded_problems["artist_over_1000"]
    artists_not_found = loaded_problems["artist_not_found"]
    no_features = loaded_problems["no_features"]

Untersuchen der aufgetretenen Probleme.

In [34]:
print(f"Insgesamt wurden für {len(no_features)} Songs keine Features gefunden.")

# Überprüfen, ob die Songs wirklich über keine Features verfügen
test_features = sp.audio_features(no_features)
songs_with_features = [item for item in test_features if item != None]
if len(songs_with_features):
  print(songs_with_features)

Insgesamt wurden für 29 Songs keine Features gefunden.


In [35]:
artists = chart_power_df_explode["Artist"].unique()
print(f"{len(artists_not_found)} von {len(artists)} Künstlern wurden von der Spotify API nicht gefunden.")

156 von 944 Künstlern wurden von der Spotify API nicht gefunden.


Eventuell sind die Lieder der Künstler in anderen Jahren erschienen.

In [211]:
artist_nfo = []
for artist in artists_not_found:
  res = sp.search(q=f"artist:{artist}", type="track", market="DE")
  if len(res["tracks"]["items"]) == 0:
    print(f"Artist {artist} not found overall.")
    artist_nfo.append(artist)

Artist b.a. robertson not found overall.
Artist the world's famous supreme team not found overall.
Artist anete humpe not found overall.
Artist t'pau not found overall.
Artist dexys midnight runners with the emerald express not found overall.
Artist terence trent d'arby not found overall.
Artist patrick cowley feat. sylvester not found overall.
Artist the christians, holly johnson, paul mccartney, gerry marsden not found overall.
Artist tommy piper singt alf not found overall.
Artist koreana not found overall.
Artist michael jackson with siedah garrett not found overall.
Artist m.a.r.r.s. not found overall.
Artist barbara gaskin not found overall.
Artist mary roos und david hanselmann not found overall.
Artist g'race not found overall.
Artist band für afrika not found overall.
Artist mike oldfield feat. aled jones, anita hegerland not found overall.
Artist technotronic feat. felly not found overall.
Artist tommi ohrner not found overall.
Artist mike oldfield and roger chapman not found

In [215]:
print(f"Insgesamt {len(artist_nfo)} Künstler wurden auch für andere Jahre nicht von der Spotify API gefunden.")
print(f"{len(artists_not_found) - len(artist_nfo)} Künstler haben Lieder in anderen Jahren veröffentlicht. Es ist jedoch unklar, ob es sich hierbei um besagte Chartsongs handelt.")

Insgesamt 31 Künstler wurden auch für andere Jahre nicht von der Spotify API gefunden.
125 Künstler haben Lieder in anderen Jahren veröffentlicht. Es ist jedoch unklar, ob es sich hierbei um besagte Chartsongs handelt.


In [36]:
df = df.applymap(lambda s: s.lower() if type(s) == str else s)

In [44]:
def convert_artist_names(df, logs = False):
  '''

  '''
  df_artists = df.copy()
  if logs:
    print('Occurences of substring ", ": ' + str(len(df_artists.loc[df_artists["artists"].str.contains(', ')])))
    print('Occurences of substring "_":' + str(len(df_artists.loc[df_artists["artists"].str.contains('_')])))

  df_artists.loc[df_artists["artists"].str.contains(', '), 'artists'] = df_artists[df_artists["artists"].str.contains(', ')]['artists'].str.replace(', ', '_')

  if logs:
    print('Occurences of substring ", ": ' + str(len(df_artists.loc[df_artists["artists"].str.contains(', ')])))
    print('Occurences of substring "_":' + str(len(df_artists.loc[df_artists["artists"].str.contains('_')])))

  df_artists['artists'] = df_artists['artists'].str.split(',')
  df_split_artists = df_artists.explode('artists')

  if logs:
    print('Occurences of substring ", ": ' + str(len(df_split_artists.loc[df_split_artists["artists"].str.contains(', ')])))
    print('Occurences of substring "_":' + str(len(df_split_artists.loc[df_split_artists["artists"].str.contains('_')])))

  df_split_artists.loc[df_split_artists['artists'].str.contains('_'), 'artists'] = df_split_artists[df_split_artists["artists"].str.contains('_')]['artists'].str.replace('_', ', ')

  if logs:
    print('Occurences of substring ", ": ' + str(len(df_split_artists.loc[df_split_artists["artists"].str.contains(', ')])))
    print('Occurences of substring "_":' + str(len(df_split_artists.loc[df_split_artists["artists"].str.contains('_')])))
  return df_split_artists

In [45]:
df_split_artists = convert_artist_names(df)
chart_artists = list(chart_power_df_explode["Artist"].unique())
unique_artists_in_dataframe = set(df_split_artists["artists"])
artists_not_in_dataframe = [artist for artist in chart_artists if artist not in unique_artists_in_dataframe]

In [46]:
len(artists_not_in_dataframe),len(chart_artists)

(288, 944)

In [49]:
dfa = df.copy()
dfa["artists"] = dfa["artists"].str.split(",")
dfa = dfa.explode("artists")

In [50]:
dfa.shape, df_split_artists.shape, df.shape

((81605, 25), (81176, 25), (68371, 25))

In [52]:
uaid = set(dfa["artists"])
anid = [artist for artist in chart_artists if artist not in uaid]
len(anid),len(chart_artists)

(287, 944)

In [60]:
chart_songs = list(chart_power_df_explode["Song"])
chart_artists = list(chart_power_df_explode["Artist"])
#zip(chart_songs, )
print(len(chart_songs), len(chart_artists))
chart_songs_artists = list(zip(chart_songs, chart_artists))

df_songs = list(df["name"])
df_artists = list(df["artists"])
df_songs_artists = list(zip(df_songs, df_artists))

2027 2027


Suchen nach Songs aus den Charts, welche nicht im DataFrame sind.

In [70]:
df_songs_artists = list(zip(df_songs, df_artists))
not_in_df = [[song, artist] for song, artist in chart_songs_artists if (song, artist) not in df_songs_artists]
print(f"Insgesamt sind {len(not_in_df)} von {len(chart_songs_artists)} Songs nicht im DataFrame enthalten")

Insgesamt sind 1153 von 2027 Songs nicht im DataFrame enthalten


In [87]:
def in_df(s, a):
  exist = False
  for song, artist in df_songs_artists:
    if (str(artist) == str(a)) and (str(s) in str(song)):
        exist = True

  return exist

not_in_df2 = [[song, artist] for song, artist in chart_songs_artists if not in_df(song, artist)]
print(f"Insgesamt sind {len(not_in_df2)} von {len(chart_songs_artists)} Songs nicht im DataFrame enthalten")

Insgesamt sind 917 von 2027 Songs nicht im DataFrame enthalten


In [ ]:
!pip install levenshtein

In [110]:
import Levenshtein

def find_closest_string(target, string_list):
    closest_string = None
    min_distance = float('inf')
    closest_index = -1

    for index, string in enumerate(string_list):
        distance = Levenshtein.distance(target, string)
        if distance < min_distance:
            min_distance = distance
            closest_string = string
            closest_index = index

    return closest_string, min_distance, closest_index

# Example usage
target_string = "absolute beginners"
string_list = [
    "beginners",
    "absolute wodka",
    "absolute beginners -"
    ]
closest, distance, index = find_closest_string(target_string, string_list)

print(f"The closest string to '{target_string}' is '{closest}' with a Levenshtein distance of {distance} and index of {index}")

The closest string to 'absolute beginners' is 'absolute beginners -' with a Levenshtein distance of 2 and index of 2


In [112]:
df.head(1)

,name,artists,album,release_date,release_date_precision,spotify_id,chart_power,uri,popularity,genres,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,isrc
0,19 - destruction mix,paul hardcastle,paul hardcastle,1985-01-01,day,1azifmcvw3orhfmdxmcro1,NaN,spotify:track:1azifmcvw3orhfmdxmcro1,35,NaN,...,1,0.0816,0.00296,0.62,0.0501,0.729,117.859,428667,4,gbayk8500158


In [96]:
df_copy = df.copy()

def in_df(s, a):
  similar_songs = []
  for song, artist in df_songs_artists:
    if ((str(artist) in str(a)) or str(a) in str(artist)) and ((str(s) in str(song)) or (str(song) in str(s))):
      #if (artist != a) or (s != song):
      #  print(artist, a)
      #  print(song, s)
      #  print("##########")
      #return True
      if (s != song):
        similar_songs.append([song, artist])
  closest, distance, index = find_closest_string(s, [item[0] for item in similar_songs])
  best_song = similar_songs[index]
  df_copy.loc[(df["name"] == best_song[0]) and (df["artists"] == best_song[1])]
  return True if closest else False

not_in_df2 = [[song, artist] for song, artist in chart_songs_artists if not in_df(song, artist)]
print(f"Insgesamt sind {len(not_in_df2)} von {len(chart_songs_artists)} Songs nicht im DataFrame enthalten")

erich kunzel,cincinnati pops orchestra,janet stubbs,sandra graham,kimberly barber,eileen farrell sandra
maria (i'll never be) maria magdalena
##########
bill medley,jennifer warnes bill medley
(i've had) the time of my life - from "dirty dancing" soundtrack (i've had) the time of my life
##########
bill medley,jennifer warnes jennifer warnes
(i've had) the time of my life - from "dirty dancing" soundtrack (i've had) the time of my life
##########
sheila e. sheila e
a love bizarre a love bizarre
##########
quincy jones,charles may quincy jones
ai no corrida ai no corrida (i-no-ko-ree-da)
##########
quincy jones,charles may quincy jones
ai no corrida ai no corrida (i-no-ko-ree-da)
##########
rufus,chaka khan rufus
ain't nobody ain't nobody
##########
rufus,chaka khan rufus
ain't nobody (live) ain't nobody
##########
rufus,chaka khan chaka khan
ain't nobody ain't nobody
##########
rufus,chaka khan chaka khan
ain't nobody (live) ain't nobody
##########
julio iglesias,diana ross diana ross


In [226]:
res = sp.search(q="artist:van halen year:1980-1989", type="track", market="DE")

In [77]:
[item["name"] for item in res["tracks"]["items"]]

NameError: ignored

In [229]:
not_in_df[:10]

[[19, 'paul hardcastle'],
 ["'cause you're young", 'c. c. catch'],
 ['(carmen) danger in her eyes', 'deborah sasson'],
 ['(carmen) danger in her eyes', 'mcl'],
 ['(do) the hucklebuck', 'coast to coast'],
 ['(feels like) heaven', 'fiction factory'],
 ['(forever) live and die', 'o.m.d.'],
 ['(hey you) the rock steady crew', 'the rock steady crew'],
 ['(i just) died in your arms', 'cutting crew'],
 ["(i'll never be) maria magdalena", 'sandra']]

Anpassen der Query. Es wird nur nach Songnamen + Künstlernamen gefiltert. Das Veröffentlichungsjahr wird nicht beachtet.

In [115]:
# Initialize offset and retries
offset = 0

artist_over_1000 = []
no_tracks = []
no_features = []
artists_not_found = []

def req_tot_results(song, artist):
  result = sp_request(lambda song, artist: sp.search(q=f"artist:{artist} track:{song}", type="track", market="DE"), 0, 100, song, artist)
  tracks = result['tracks'] if 'tracks' in result else ''
  return tracks['total'] if 'total' in tracks else 0


df = pd.DataFrame()
with tqdm(total=len(not_in_df)) as pbar:
  tracks_ = []
  audio_features_ = []

  for song, artist in not_in_df:
    total_results = req_tot_results(song, artist)
    if total_results == 0:
      print(f"{artist}, {song} not found!")
      artists_not_found.append([artist, song])
    elif total_results < 1000:
      offset = 0
      while offset < total_results:
        result = sp_request(lambda artist, offset, song: sp.search(q=f"artist:{artist} track:{song}", type="track", limit=50, offset=offset, market="DE"), 0, 100, artist, offset, song)
        offset += 50
        if "tracks" not in result:
          print("No tracks in result")
          continue

        tracks = result["tracks"]

        if "items" in tracks and isinstance(tracks["items"], list):
          for track in tracks["items"]:
            tracks_.append(track)
            if len(tracks_) == 100:
              audio_features_ = sp_request(lambda x: sp.audio_features([track["id"] for track in x]), 0, 100, tracks_)
              track_features = []
              if len(tracks_) != len(audio_features_):
                print("Tracks and features have not the same lengh!")
                print(len(tracks_), len(audio_features_))
              for t, af in list(zip(tracks_, audio_features_)):
                if not t:
                  print("No track!")
                  print(t)
                  print(af)
                  no_tracks.append(af)
                  continue
                if not af:
                  print("No features")
                  print(af)
                  print(t)
                  no_features.append(t)
                  continue
                features = filter_track_features(t, af, False)
                track_features.append(features)
              df = pd.concat([df, pd.DataFrame(track_features)], ignore_index=True)
              #df.to_csv(f"{PATH}/data-missing.csv")
              tracks_ = []
              audio_features_ = []
        else:
          continue
    else:
      print(f"Artist {artist} has published over 1000 songs in the 80s")
      artist_over_1000.append(artist)

    pbar.update(1)

problems = {
    "artist_over_1000": artist_over_1000,
    "artist_not_found": artists_not_found,
    "no_features": [track["id"] for track in no_features]
}
#with open(f"{PATH}/data-missing-problems.json", "w") as file:
#    json.dump(problems, file)

  0%|          | 3/1153 [00:01<06:54,  2.78it/s]

mcl, (carmen) danger in her eyes not found!


  0%|          | 5/1153 [00:01<04:53,  3.91it/s]

o.m.d., (forever) live and die not found!


  1%|          | 9/1153 [00:02<03:13,  5.92it/s]

...

 98%|█████████▊| 1133/1153 [20:02<00:30,  1.52s/it]

heidi brühl, you are a part of my heart not found!


 99%|█████████▉| 1142/1153 [20:07<00:05,  2.04it/s]

jona lewie, you'll always find me in the kitchen at parties not found!
peter kent, you're all i need not found!


100%|██████████| 1153/1153 [20:10<00:00,  1.05s/it]


In [151]:
df = pd.read_csv(f"{PATH}/data-missing.csv")

Analysieren der Ergebnisse:

In [78]:
df.shape

(68371, 25)

Es wurden ~21.000 Songs erfragt bei gerade einmal 1153 Kombinationen. Eventuell macht es doch Sinn, das Jahr zu reduzieren. Es werden alle Songs ab dem Jahr 1990 herausgefiltert, da diese nicht in den Charts der 80er existieren konnten.

In [153]:
# Erstellen eines neuen Features "year"
df["year"] = df["release_date"].apply(lambda x: int(x[:4]))

In [154]:
df = df[df.year < 1990]
df.shape

(1438, 26)

In [155]:
df.head()

,Unnamed: 0,name,artists,album,release_date,release_date_precision,spotify_id,chart_power,uri,popularity,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,isrc,year
1,1,19 - Destruction Mix,Paul Hardcastle,Paul Hardcastle,1985-01-01,day,1AzIfmCVW3orhfmdxmCRo1,NaN,spotify:track:1AzIfmCVW3orhfmdxmCRo1,35,...,0.0816,0.00296,0.62000,0.0501,0.729,117.859,428667,4,GBAYK8500158,1985
3,3,19 - New Version,Paul Hardcastle,Paul Hardcastle,1985-01-01,day,6B2Tn5vzAFhJUuBG3N77op,NaN,spotify:track:6B2Tn5vzAFhJUuBG3N77op,22,...,0.0681,0.00615,0.24600,0.4120,0.362,117.901,378240,4,GBAYK0700009,1985
5,5,19 - The Final Story,Paul Hardcastle,Paul Hardcastle,1985-01-01,day,6vLlYsgxxQCynGiEkcgELZ,NaN,spotify:track:6vLlYsgxxQCynGiEkcgELZ,17,...,0.0873,0.00133,0.12100,0.2800,0.626,117.971,286933,4,GBAYK0700168,1985
10,10,19 - The Final Story Requiem,Paul Hardcastle,Paul Hardcastle,1985-01-01,day,4yEyIFA2DhKbDIOMB0dOBF,NaN,spotify:track:4yEyIFA2DhKbDIOMB0dOBF,13,...,0.0964,0.04680,0.08860,0.4200,0.545,116.874,185560,4,GBAYK0700169,1985
74,74,Earth from Space / Ground Zero / No Winners / ...,Paul Hardcastle,No Winners,1988-01-01,day,0XrnWmiSCL5TJtclcfFvwM,NaN,spotify:track:0XrnWmiSCL5TJtclcfFvwM,4,...,0.0690,0.04760,0.00358,0.3350,0.298,120.022,1889840,4,GBAYK0800041,1988


Dies hat die Anzahl der Songs von 21098 auf 1438 reduziert.

In [156]:
df = df.applymap(lambda s: s.lower() if type(s) == str else s)
df_songs = list(df["name"])
df_artists = list(df["artists"])
df_songs_artists = list(zip(df_songs, df_artists))

In [157]:
not_in_df = [[song, artist] for song, artist in chart_songs_artists if (song, artist) not in df_songs_artists]
print(f"Insgesamt sind {len(not_in_df)} von {len(chart_songs_artists)} Songs nicht im DataFrame enthalten")

Insgesamt sind 1978 von 2027 Songs nicht im DataFrame enthalten


In [217]:
res = sp.search(q="Zob Marley", type="track", market="DE")

In [219]:
res["tracks"]["items"]

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0KYZ0U8VUXgIscN7Scodd2'},
     'href': 'https://api.spotify.com/v1/artists/0KYZ0U8VUXgIscN7Scodd2',
     'id': '0KYZ0U8VUXgIscN7Scodd2',
     'name': 'Jah Marley',
     'type': 'artist',
     'uri': 'spotify:artist:0KYZ0U8VUXgIscN7Scodd2'}],
   'external_urls': {'spotify': 'https://open.spotify.com/album/7gLQR5W5eqZYKZRQHRRQTi'},
   'href': 'https://api.spotify.com/v1/albums/7gLQR5W5eqZYKZRQHRRQTi',
   'id': '7gLQR5W5eqZYKZRQHRRQTi',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b27338a5a4c647dc803c1cc45ff6',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn.co/image/ab67616d00001e0238a5a4c647dc803c1cc45ff6',
     'width': 300},
    {'height': 64,
     'url': 'https://i.scdn.co/image/ab67616d0000485138a5a4c647dc803c1cc45ff6',
     'width': 64}],
   'is_playable': True,
   'name': 'Solitaire',
   'release_date': '2018-12-01',

In [158]:
df.head()

,Unnamed: 0,name,artists,album,release_date,release_date_precision,spotify_id,chart_power,uri,popularity,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,isrc,year
1,1,19 - destruction mix,paul hardcastle,paul hardcastle,1985-01-01,day,1azifmcvw3orhfmdxmcro1,NaN,spotify:track:1azifmcvw3orhfmdxmcro1,35,...,0.0816,0.00296,0.62000,0.0501,0.729,117.859,428667,4,gbayk8500158,1985
3,3,19 - new version,paul hardcastle,paul hardcastle,1985-01-01,day,6b2tn5vzafhjuubg3n77op,NaN,spotify:track:6b2tn5vzafhjuubg3n77op,22,...,0.0681,0.00615,0.24600,0.4120,0.362,117.901,378240,4,gbayk0700009,1985
5,5,19 - the final story,paul hardcastle,paul hardcastle,1985-01-01,day,6vllysgxxqcyngiekcgelz,NaN,spotify:track:6vllysgxxqcyngiekcgelz,17,...,0.0873,0.00133,0.12100,0.2800,0.626,117.971,286933,4,gbayk0700168,1985
10,10,19 - the final story requiem,paul hardcastle,paul hardcastle,1985-01-01,day,4yeyifa2dhkbdiomb0dobf,NaN,spotify:track:4yeyifa2dhkbdiomb0dobf,13,...,0.0964,0.04680,0.08860,0.4200,0.545,116.874,185560,4,gbayk0700169,1985
74,74,earth from space / ground zero / no winners / ...,paul hardcastle,no winners,1988-01-01,day,0xrnwmiscl5tjtclcffvwm,NaN,spotify:track:0xrnwmiscl5tjtclcffvwm,4,...,0.0690,0.04760,0.00358,0.3350,0.298,120.022,1889840,4,gbayk0800041,1988


In [159]:
with open(f"{PATH}/data-missing-problems.json", "r") as file:
  data_missing_problems = json.load(file)

In [160]:
artists_not_found = data_missing_problems["artist_not_found"]

In [164]:
len(artists_not_found)

263